# <span style="color:darkblue"> Lecture 19 - Connecting to SQL via Python </span>

<font size = "5">

In the past lecture:

- We worked directly in SQL
- Carried out basic dataset operations

In  this lecture we will run SQL from Python

- Combine the best of both!
- Learn about escape characters!

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">

Import libraries

In [1]:
# psycogpg2 helps us process SQL commands to send to the server
# sqlalchemy facilitates establishing a connection with the server

import pandas as pd
from sqlalchemy import create_engine
from   sqlalchemy.engine import URL
from   sqlalchemy import text

<font size = "5">

Connect to SQL server

- In the default instructions we set <br>
the password to "12345" in windows and <br>
no password for Mac
- ADJUST code accordingly! 

In [2]:
# Use the connection details to your server
# These are the default settings.
# "postgresql" is a fixed argument
# If you have a different host,database, username, or password,
# change the corresponding connection details


url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = "12345")

connection = create_engine(url_server)


# <span style="color:darkblue"> II. Entity Relationship Diagrams </span>

<font size = "5">

United States Congress dataset

- Bills and actions
- US congress members



<font size = "5">

Read datasets into Python

In [3]:
bills_actions        = pd.read_csv("data_raw/bills_actions.csv")
us_congress_members = pd.read_csv("data_raw/us_congress_members.csv")

<font size = "5">

Upload to SQL


In [4]:
bills_actions.to_sql('bills_actions',
               con = connection,
               if_exists='replace',
               index=False)

# Import "races"
us_congress_members.to_sql('us_congress_members', 
             con = connection, 
             if_exists='replace',
             index=False)

811

<font size = "5">

Download from SQL Server

In [5]:
# Recall that "*" indicates that we should select all columns

data  = pd.read_sql(text('SELECT * \
                          FROM bills_actions;'), connection)

#display(data)

<font size = "5">

Try it yourself

Download and display the "us_congress_members" dataset from SQL

In [7]:
# Write your own code

data_congress  = pd.read_sql(text('SELECT * \
                          FROM us_congress_members;'), connection)

display(data_congress)


,member_id,full_name,last_name,member_title,state,party_name,chamber
0,0,A. Donald McEachin,McEachin,Representative,Virginia,Democratic,House
1,1,Aaron Schock,Schock,Representative,Illinois,Republican,House
2,2,Abby Finkenauer,Finkenauer,Representative,Iowa,Democratic,House
3,3,Abigail Davis Spanberger,Spanberger,Representative,Virginia,Democratic,House
4,4,Adam H. Putnam,Putnam,Representative,Florida,Republican,House
...,...,...,...,...,...,...,...
1806,1806,Wm. Lacy Clay,Clay,Representative,Missouri,Democratic,House
1807,1807,"Wyche, Jr. Fowler",Fowler,Senator,Georgia,Democratic,House
1808,1808,"Wyche, Jr. Fowler",Fowler,Senator,Georgia,Democratic,Senate
1809,1809,Yvette D. Clarke,Clarke,Representative,New York,Democratic,House


# <span style="color:darkblue"> III. Merge Dataset </span>

<font size = "5">

The following is an example of an entity relationship diagram (ERD)



<font size = "5">

<img src="figures/rdb_us_congress.png" alt="drawing" width="650"/>

- Here "member_id" is the PRIMARY KEY of the first dataset
- We can check that with SQL commands

In [7]:
summary = pd.read_sql(text("SELECT COUNT(DISTINCT member_id) AS num_distinct, \
                           COUNT(*) AS num_members   \
                           FROM us_congress_members"),
                      connection)

summary

,num_distinct,num_members
0,1811,1811


<font size = "5">

Merge two datasets

- Similar to python
- FROM is followed by the name of the primary dataset
- LEFT is followed by the name of the secondary dataset
- ON is the id variable used for merging

In [8]:
# Write your own code

example = pd.read_sql(text("SELECT * \
                            FROM bills_actions \
                            LEFT JOIN us_congress_members \
                            ON bills_actions.member_id = us_congress_members.member_id ;"),
                      connection)

# example


# <span style="color:darkblue"> IV. (Optional) Additional Resources </span>


<font size = "5">

SQL in 100 seconds:

https://www.youtube.com/watch?v=zsjvFFKOm3c

Overview of SQL:

https://www.youtube.com/watch?v=27axs9dO7AE

Introduction to databases:

https://www.youtube.com/watch?v=wR0jg0eQsZA

Introduction to Entity Relationship Diagrams

https://www.youtube.com/watch?v=wR0jg0eQsZA


